# ABEL linac tracking example

By Carl A. Lindstrøm (University of Oslo), 8 Sep 2023

### Import ABEL framework

In [1]:
import sys
sys.path.append('../')
from abel import *
#from abel.classes.stage.impl import stage_quasistatic_2d_radiation_reaction as rr
import numpy as np
import matplotlib.pyplot as plt
from abel.utilities.statistics import prct_clean, prct_clean2d
from matplotlib.animation import FuncAnimation
import time

initializing ocelot...


### Define the linac and beams

In [2]:
# define driver
driver = SourceBasic()
driver.charge = -2.7e10 * SI.e # [C]
driver.energy = 31.25e9 # [eV]
driver.rel_energy_spread = 0.01
driver.bunch_length = 50e-6 # [m]
driver.z_offset = 580e-6 # [m]
driver.emit_nx, driver.emit_ny = 10e-6, 10e-6 # [m rad]
driver.beta_x, driver.beta_y = 30e-3, 30e-3 # [m]
driver.num_particles = 10000
driver.jitter.x = 0 # [m] 2e-6
driver.jitter.y = 0 # [m] 100e-9
driver.jitter.t = 10e-15 # [m]

# define stage
stage = StageQuasistatic2dRadiationReaction()
stage.driver_source = driver
stage.nom_energy_gain = 31.9e9 # [eV]
stage.length = 11 # [m]
stage.plasma_density = 2e21 # [m^-3]
stage.ramp_beta_mag = 5
stage.enable_rr = False

# define first stage (half length)
first_stage = StageQuasistatic2dRadiationReaction()
first_stage.driver_source = stage.driver_source
first_stage.nom_energy_gain = stage.nom_energy_gain/2
first_stage.length = stage.length/2
first_stage.plasma_density = stage.plasma_density
first_stage.ramp_beta_mag = stage.ramp_beta_mag
first_stage.enable_rr = stage.enable_rr

# define beam
source = SourceBasic()
source.charge = -1e10 * SI.e # [C]
source.energy = 5e9 # [eV]
source.rel_energy_spread = 0.01
source.bunch_length = 28e-6 # [m]
source.z_offset = 0e-6 # [m]
source.emit_nx, source.emit_ny = 160e-6, 0.56e-6 # [m rad]
source.beta_x = stage.matched_beta_function(source.energy)
source.beta_y = source.beta_x
source.num_particles = 10000
source.jitter.t = driver.jitter.t

# define interstage
interstage = InterstageBasic()
interstage.beta0 = lambda E: stage.matched_beta_function(E)
interstage.dipole_length = lambda E: 1 * np.sqrt(E/10e9) # [m(eV)]
interstage.dipole_field = 1 # [T]

# beam delivery system
bds = BeamDeliverySystemBasic()
bds.beta_x, bds.beta_y = 8e-3, 0.4e-3 # [m]
bds.bunch_length = 0.75 * bds.beta_y

# define linac
linac = Linac()
linac.source = source
linac.stage = stage
linac.first_stage = first_stage
linac.interstage = interstage
linac.num_stages = 16

### Run simulations

In [ ]:
start = time.process_time()

linac.run(f'linac_{int(source.energy/1e9)}GeV_{interstage.dipole_field}T_ex_{source.emit_nx*1e6}_jitterx_{driver.jitter.x}_jittery_{driver.jitter.y}_RR_{stage.enable_rr}', num_shots=1, overwrite=True, parallel=False);

print((time.process_time() - start)/60)

Tracking element 1 (s = 0.0 m, -1.60 nC, 5.0 GeV, SourceBasic, stage 0)


/home/daniel/.local/lib/python3.8/site-packages/aptools/data_analysis/beam_diagnostics.py:524: RuntimeWarning: invalid value encountered in sqrt
  em_x = np.sqrt(np.linalg.det(cov_x.astype(np.float32, copy=False)))


Tracking element 2 (s = 5.5 m, -1.60 nC, 21.8 GeV, StageQuasistatic2dRadiationReaction, stage 1)
Tracking element 3 (s = 12.4 m, -1.60 nC, 21.8 GeV, InterstageBasic, stage 1)
Tracking element 4 (s = 23.4 m, -1.60 nC, 53.6 GeV, StageQuasistatic2dRadiationReaction, stage 2)
Tracking element 5 (s = 34.4 m, -1.60 nC, 53.6 GeV, InterstageBasic, stage 2)
Tracking element 6 (s = 45.4 m, -1.60 nC, 85.5 GeV, StageQuasistatic2dRadiationReaction, stage 3)
Tracking element 7 (s = 59.4 m, -1.60 nC, 85.5 GeV, InterstageBasic, stage 3)
Tracking element 8 (s = 70.4 m, -1.60 nC, 116.2 GeV, StageQuasistatic2dRadiationReaction, stage 4)
Tracking element 9 (s = 86.7 m, -1.60 nC, 116.2 GeV, InterstageBasic, stage 4)
Tracking element 10 (s = 97.7 m, -1.60 nC, 148.8 GeV, StageQuasistatic2dRadiationReaction, stage 5)
Tracking element 11 (s = 116.2 m, -1.60 nC, 148.8 GeV, InterstageBasic, stage 5)
Tracking element 12 (s = 127.2 m, -1.60 nC, 180.8 GeV, StageQuasistatic2dRadiationReaction, stage 6)
Tracking elem

In [ ]:
#beam2 = linac.run('linac_5TeV', num_shots=1, overwrite=False, parallel=False);

### Plot the wakefield

In [ ]:
# plot the first-stage wakefield
linac.stages[0].plot_wakefield()

In [ ]:
# plot the last-stage wakefield
linac.stages[-1].plot_wakefield()

### Plot beam evolution and survey

In [ ]:
# plot linac survey
linac.plot_survey()

# plot beam evolution
fig_9GeV = linac.plot_evolution(use_stage_nums=False)
#fig_9GeV.savefig(f'Plots_with_updated_params/{int(source.energy/1e9)}GeV_{int(source.emit_nx*1e6)}mmmrad_{int(source.emit_ny*1e6)}\
#mmmrad_{interstage.dipole_field}T_RR_{stage.enable_rr}_B_{interstage.dipole_field}T_deleted_low_z_values_10k.png')

linac.plot_waterfalls()

In [ ]:
#linac.animate_lps(file_name = f'{interstage.dipole_field}T_5GeV_ex_{source.emit_nx*1e6}_RR_{stage.enable_rr}_jitterx_{driver.jitter.x*1e9}_jittery_{driver.jitter.y*1e9}_nm_weird_results')